<a href="https://colab.research.google.com/github/oluwafemidiakhoa/AIreasearcher/blob/main/CRAssistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Install Required Libraries
Install the necessary libraries for the system:

In [ ]:
!pip install streamlit transformers datasets openai torch plotly reportlab


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not c

## Step 2: Load and Preprocess Dataset
Use the Hugging Face Datasets library to load and preprocess clinical research data.

Load Dataset

In [ ]:
from datasets import load_dataset

# Load clinical dataset (e.g., scicite for citation intent classification)
dataset = load_dataset("scicite")
train_data = dataset["train"]
test_data = dataset["test"]


README.md:   0%|          | 0.00/9.06k [00:00<?, ?B/s]

scicite.py:   0%|          | 0.00/6.23k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/3.10k [00:00<?, ?B/s]

The repository for scicite contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/scicite.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/8194 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/916 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1859 [00:00<?, ? examples/s]

## Preprocess Data

In [ ]:
from transformers import AutoTokenizer

# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Define a function to tokenize text data
def preprocess_data(examples):
    return tokenizer(examples["string"], truncation=True, padding="max_length", max_length=512)

# Tokenize the train and test datasets
encoded_train = train_data.map(preprocess_data, batched=True)
encoded_test = test_data.map(preprocess_data, batched=True)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/8194 [00:00<?, ? examples/s]

Map:   0%|          | 0/1859 [00:00<?, ? examples/s]

## Step 3: Fine-Tune a Model
Fine-tune a Hugging Face model for text classification and save it for reuse.

Define and Train the Model

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load a pre-trained model with a classification head
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_test,
    tokenizer=tokenizer
)

# Train the model
trainer.train()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-4-a694c4b5a4f4>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wand

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.426800,0.417131
2,0.291100,0.599938
3,0.193100,0.662406


TrainOutput(global_step=3075, training_loss=0.3322867060095314, metrics={'train_runtime': 603.3884, 'train_samples_per_second': 40.74, 'train_steps_per_second': 5.096, 'total_flos': 6467854034589696.0, 'train_loss': 0.3322867060095314, 'epoch': 3.0})

## Save the Model
Save the fine-tuned model to a local directory for reuse:

In [ ]:
# Save the model and tokenizer
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")
print("Model and tokenizer saved locally at './saved_model'")


Model and tokenizer saved locally at './saved_model'


## Upload the trained model and tokenizer to Hugging Face later using the following script:

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Set Hugging Face credentials
import os
os.environ["HF_USERNAME"] = "mgbam"
os.environ["HF_TOKEN"] = ""

# Reload the saved model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("./saved_model")
tokenizer = AutoTokenizer.from_pretrained("./saved_model")

# Push the model and tokenizer to Hugging Face Hub
model.push_to_hub("bert-base-finetuned-mgbam", use_auth_token=os.environ["HF_TOKEN"])
tokenizer.push_to_hub("bert-base-finetuned-mgbam", use_auth_token=os.environ["HF_TOKEN"])

print("Model and tokenizer uploaded to Hugging Face Hub!")


/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Model and tokenizer uploaded to Hugging Face Hub!


## Step 4: Create a Gradio Interface

In [ ]:
!pip install gradio PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.7 MB/s eta 0:00:00


## Step 5: Add Functionalities
1. Summarization Using OpenAI

In [ ]:
!pip install gradio openai transformers torch reportlab matplotlib huggingface-hub loguru



## Setup Prediction Using Hugging Face Model



In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()  # Load variables from .env file if it exists

import gradio as gr
from openai import OpenAI
import torch
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    MarianMTModel,
    MarianTokenizer,
)
from huggingface_hub import login
from reportlab.pdfgen import canvas
import matplotlib.pyplot as plt
import io
from PIL import Image
from loguru import logger
import spacy
import spacy.cli
from PyPDF2 import PdfReader
import altair as alt
from google.colab import userdata


# Ensure SpaCy language model is downloaded
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Downloading SpaCy 'en_core_web_sm' model...")
    spacy.cli.download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")

# Configure logging
logger.add("error_logs.log", rotation="1 MB", level="ERROR")

# Retrieve credentials from environment variables
HUGGINGFACE_TOKEN = userdata.get("HF_TOKEN")
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

if not HUGGINGFACE_TOKEN or not OPENAI_API_KEY:
    raise ValueError("Missing Hugging Face or OpenAI API credentials in environment variables.")

# Log in to Hugging Face Hub
login(HUGGINGFACE_TOKEN)

# Initialize OpenAI Client
client = OpenAI(api_key=OPENAI_API_KEY)

# Load Hugging Face classification model and tokenizer
model_name = "mgbam/bert-base-finetuned-mgbam"
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, token=HUGGINGFACE_TOKEN
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name, token=HUGGINGFACE_TOKEN
)

# Load translation model and tokenizer
translation_model = MarianMTModel.from_pretrained(
    "Helsinki-NLP/opus-mt-en-fr", token=HUGGINGFACE_TOKEN
)
translation_tokenizer = MarianTokenizer.from_pretrained(
    "Helsinki-NLP/opus-mt-en-fr", token=HUGGINGFACE_TOKEN
)

# Supported languages for translation
LANGUAGE_MAP = {
    "English to French": ("en", "fr"),
    "French to English": ("fr", "en"),
    "English to Spanish": ("en", "es"),
    "Spanish to English": ("es", "en"),
    "English to German": ("en", "de"),
    "German to English": ("de", "en"),
}

# Summarization Function
def summarize_text(text):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": f"Summarize the following clinical data:\n{text}"}],
            max_tokens=200,
            temperature=0.7,
        )
        return response.choices[0].message.content
    except Exception as e:
        logger.error(f"Summarization Error: {e}")
        return "Summarization failed. Please try again."

# Prediction Function
def predict_outcome(text):
    try:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1).tolist()[0]
        return {f"Label {i+1}": round(pred, 3) for i, pred in enumerate(predictions)}
    except Exception as e:
        logger.error(f"Prediction Error: {e}")
        return "Prediction failed. Please try again."

# Report Generation Function
def generate_report(text, filename="clinical_report.pdf"):
    try:
        c = canvas.Canvas(filename)
        c.drawString(100, 750, "Clinical Research Report")
        c.drawString(100, 730, f"Input Text: {text}")
        c.save()
        return filename
    except Exception as e:
        logger.error(f"Report Generation Error: {e}")
        return None

# Visualization Function
def visualize_predictions(text):
    try:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1).tolist()[0]
        labels = [f"Label {i+1}" for i in range(len(predictions))]

        data = pd.DataFrame({"Labels": labels, "Probabilities": predictions})
        chart = alt.Chart(data).mark_bar().encode(
            x="Labels",
            y="Probabilities",
            color="Labels"
        ).properties(width=600, height=400)
        return chart  # Return the Altair chart directly
    except Exception as e:
        logger.error(f"Visualization Error: {e}")
        return None

# Translation Function
def translate_text(text, translation_option):
    try:
        src_lang, tgt_lang = LANGUAGE_MAP[translation_option]
        inputs = translation_tokenizer(text, return_tensors="pt", padding=True)
        translated = translation_model.generate(**inputs)
        return translation_tokenizer.decode(translated[0], skip_special_tokens=True)
    except Exception as e:
        logger.error(f"Translation Error: {e}")
        return "Translation failed. Please try again."

# File Processing Function
def process_file(file_path):
    try:
        if file_path.endswith(".pdf"):
            reader = PdfReader(file_path)
            text = "".join(page.extract_text() for page in reader.pages)
            return summarize_text(text)
        elif file_path.endswith(".csv"):
            df = pd.read_csv(file_path)
            return df.head().to_string()
        else:
            return "Unsupported file format. Please upload a PDF or CSV."
    except Exception as e:
        logger.error(f"File Processing Error: {e}")
        return "Error processing the file."

# NLP Analysis Function
def analyze_entities(text):
    try:
        doc = nlp(text)
        entities = [{"text": ent.text, "label": ent.label_} for ent in doc.ents]
        return entities
    except Exception as e:
        logger.error(f"NLP Analysis Error: {e}")
        return "Entity recognition failed."

# Gradio Interface
if __name__ == "__main__":
    with gr.Blocks() as demo:
        gr.Markdown("# AI-Powered Clinical Research Assistant")

        # Main Input and Actions
        text_input = gr.Textbox(lines=5, placeholder="Enter clinical text or query here...", label="Clinical Text")
        action = gr.Radio(
            ["Summarize", "Predict Outcome", "Generate Report", "Visualize Predictions", "Translate", "Entity Recognition", "Upload File"],
            label="Select an Action"
        )

        # Translation Option
        translation_option = gr.Dropdown(
            choices=list(LANGUAGE_MAP.keys()),
            label="Translation Option (Required for Translate)",
            visible=False
        )

        # File Upload
        file_input = gr.File(label="Upload Clinical Document", type="filepath", visible=False)

        # Outputs
        output_text = gr.Textbox(label="Output")
        output_chart = gr.Plot(label="Visualization")
        output_file = gr.File(label="Generated Report")

        # Show/Hide Components
        def toggle_components(action):
            return {
                translation_option: gr.update(visible=action == "Translate"),
                file_input: gr.update(visible=action == "Upload File"),
            }

        action.change(toggle_components, inputs=[action], outputs=[translation_option, file_input])

        # Submit Button
        submit_button = gr.Button("Submit")

        def handle_action(action, text, translation_option=None, file=None):
            if action == "Summarize":
                return summarize_text(text), None, None
            elif action == "Predict Outcome":
                return predict_outcome(text), None, None
            elif action == "Generate Report":
                return None, None, generate_report(text)
            elif action == "Visualize Predictions":
                return None, visualize_predictions(text), None
            elif action == "Translate":
                if not translation_option:
                    return "Please select a translation option.", None, None
                return translate_text(text, translation_option), None, None
            elif action == "Entity Recognition":
                return analyze_entities(text), None, None
            elif action == "Upload File":
                return process_file(file), None, None
            return "Invalid action.", None, None

        submit_button.click(
            handle_action,
            inputs=[action, text_input, translation_option, file_input],
            outputs=[output_text, output_chart, output_file],
        )

    demo.launch(share=True)
